##Initialize the Spark and SparkSQL contexts

In [ ]:
library(SparkR)
sc <- sparkR.init("local[*]")
sqlCtx<- sparkRSQL.init(sc)

##Load the 3 datasets from Parquet files into SparkR as DataFrames
- **txnsRaw**: Transaction data organized by line-item
- **demo**: Demographic data, organized by customer ID
- **sample**: A subset of customers who received a DM offer. This is our target variable.

In [ ]:
txnsRaw<- loadDF(sqlCtx, paste(getwd(), "/Customer_Transactions.parquet", sep = ""), "parquet")
demo <- withColumnRenamed(loadDF(sqlCtx, paste(getwd(), "/Customer_Demographics.parquet", sep = ""), "parquet"),
                          "cust_id", "ID")
sample <- loadDF(sqlCtx, paste(getwd(), "/DM_Sample.parquet", sep = ""), "parquet")

##We can view the schema of any DataFrame with `printSchema()`

In [ ]:
printSchema(txnsRaw)

##Now we need to generate a few measures for customer behavior to use in our model.

###Using our transactions data, we can create the follow variables:
- **txns**: The number of transactions per customer
- **spend**: Total expenditure per customer

###To do this, we'll use the `groupBy` and `agg` functions:

In [ ]:
perCustomer <- agg(groupBy(txnsRaw,"cust_id"),
                   txnsRaw$cust_id,
                   txns = countDistinct(txnsRaw$day_num),
                   spend = sum(txnsRaw$extended_price))

head(perCustomer)

##We also want to include demographic information about each customer in our model, so we'll need to join `perCustomer` to `demo` and select the relevant fields:

In [ ]:
joinToDemo <- select(join(perCustomer, demo),
                     demo$"*",
                     perCustomer$txns, 
                     perCustomer$spend)

##Since we're using SparkSQL, our commands are being optimized by the Catalyst query optimizer. Let's take a look at Catalyst's plan for `joinToDemo`.

In [ ]:
explain(joinToDemo)

##Wait, a Cartesian Product?

In [ ]:
joinToDemo <- select(join(perCustomer, demo, perCustomer$cust_id == demo$ID),
                     demo$"*",
                     perCustomer$txns, 
                     perCustomer$spend)

explain(joinToDemo)

##Now that we've got all our variables prepared, we need to create separate training and estimation sets.

In [ ]:
trainDF <- select(join(joinToDemo, sample, joinToDemo$ID == sample$cust_id),
                joinToDemo$"*",
                alias(cast(sample$respondYes, "double"), "respondYes"))

estDF <- select(filter(join(joinToDemo, sample, joinToDemo$ID == sample$cust_id, "left_outer"),
                      "cust_id IS NULL"),
               joinToDemo$"*")

printSchema(estDF)

##Now that we've got our data prepped and pared down, we can turn each SparkSQL DataFrame into an R `data.frame`:

In [ ]:
train <- collect(trainDF) ; train$ID <- NULL

est <- collect(estDF)

In [ ]:
class(est)

##Since we can seamlessly transition from Spark to R, we can make full use of SparkSQL's distributed query features while prepping our data and then switch to using R's native modeling features:

In [ ]:
theModel <- glm(respondYes ~ ., "binomial", train)

summary(theModel)

##Finally, let's create a custom scoring function that will use R's `predict` method and also output the scores by customer ID.

In [ ]:
predictWithID <- function(modObj, data, idField) {
  scoringData <- data[, !names(data) %in% as.character(idField)]
  scores <- predict(modObj, scoringData, type = "response", se.fit = TRUE)
  idScores <- data.frame(ID = data[as.character(idField)], Score = scores$fit)
  idScores[order( -idScores$Score), ]
}

testScores <- predictWithID(theModel, est, "ID")

In [ ]:
head(testScores, 25)